In [1]:
import json
import math as m

def cargo_volume_m3(cargo_json):
    #m = cargo_json.get('margin', { 'length': 0, 'width': 0, 'height': 0 })
    m = { 'length': 0, 'width': 0, 'height': 0 }
    a = (float(cargo_json['size']['length'])
        + 2.0 * float(m['length'])) * 1E-3
    b = (float(cargo_json['size']['width'])
        + 2.0 * float(m['width'])) * 1E-3
    c = (float(cargo_json['size']['height'])
        + 2.0 * float(m['height'])) * 1E-3
    return a*b*c

def limit_volume_m3(cargo_space_json):
    m = cargo_space_json.get('margin', { 'length': 0, 'width': 0, 'height': 0 })
    a = (float(cargo_space_json['loading_size']['length'])
        - 2.0 * float(m['length'])) * 1E-3
    b = (float(cargo_space_json['loading_size']['width'])
        - 2.0 * float(m['width'])) * 1E-3
    c = (float(cargo_space_json['loading_size']['height'])
        - 2.0 * float(m['height'])) * 1E-3
    indentation = cargo_space_json['indentation']
    a_indent = (float(indentation['length']) + float(indentation.get('length_end', 0))) * 1E-3
    b_indent = (float(indentation['width']) + float(indentation.get('width_right', 0))) * 1E-3
    c_indent = (float(indentation['height']) + float(indentation.get('height_top', 0))) * 1E-3
    a += a_indent
    b += b_indent
    c += c_indent
    return a*b*c

def total_cargo_count(cargoes_array_json):
    count = 0
    for cargo in cargoes_array_json:
        count+=cargo['count']
    return count

def unstackable_cargo_count(cargoes_array_json):
    count = 0
    for cargo in cargoes_array_json:
        if not bool(cargo.get('stacking', True)):
            count+=cargo['count']
    return count

def unturnoverable_cargo_count(cargoes_array_json):
    count = 0
    for cargo in cargoes_array_json:
        if not bool(cargo.get('turnover', True)):
            count+=cargo['count']
    return count

def min_cargo_volume_m3(cargoes_array_json):
    return min([cargo_volume_m3(cargo) for cargo in cargoes_array_json])

def max_cargo_volume_m3(cargoes_array_json):
    return max([cargo_volume_m3(cargo) for cargo in cargoes_array_json])

def weighted_avg_cargo_volume_m3(cargoes_array_json):
    s = 0.0
    ws = 0.0
    for cargo in cargoes_array_json:
        s += cargo_volume_m3(cargo)*float(cargo['count'])
        ws += float(cargo['count'])
    return s/ws

def cargo_mass(cargo_json):
    mass = float(cargo_json['mass'])
    vol = cargo_volume_m3(cargo_json)
    density = mass / vol
    if density <= 20000.0:
        return mass
    else:
        return 750.0 * vol

def min_cargo_mass(cargoes_array_json):
    return min([cargo_mass(cargo) for cargo in cargoes_array_json])

def max_cargo_mass(cargoes_array_json):
    return max([cargo_mass(cargo) for cargo in cargoes_array_json])

def total_cargo_mass(cargoes_array_json):
    return sum([cargo_mass(cargo) for cargo in cargoes_array_json])

def weighted_avg_cargo_mass(cargoes_array_json):
    s = 0.0
    ws = 0.0
    for cargo in cargoes_array_json:
        s += float(cargo_mass(cargo))*float(cargo['count'])
        ws += float(cargo['count'])
    return s/ws

def total_cargo_volume_m3(cargoes_array_json):
    return sum([cargo_volume_m3(cargo)*float(cargo['count']) for cargo in cargoes_array_json])

def total_cargo_mass(cargoes_array_json):
    return sum([cargo_mass(cargo)*float(cargo['count']) for cargo in cargoes_array_json])

def filling_space_percent(cargoes_data_json):
    return float(cargoes_data_json['data_result']['cargo_space']['calculation_info']['filling_space_percent'])

def estimated_space_percent(cargoes_data_json):
    cargo_vol = total_cargo_volume_m3(cargoes_data_json['data']['cargoes'])
    limit_vol = limit_volume_m3(cargoes_data_json['data']['cargo_space'])
    cargo_mass = total_cargo_mass(cargoes_data_json['data']['cargoes'])
    limit_mass = float(cargoes_data_json['data']['cargo_space']['carrying_capacity'])
    avg_cargo_vol = weighted_avg_cargo_volume_m3(cargoes_data_json['data']['cargoes'])
    avg_cargo_mass = weighted_avg_cargo_mass(cargoes_data_json['data']['cargoes'])
    if cargo_vol > limit_vol or cargo_mass > limit_mass:
        cargo_pos_count = min(m.floor(limit_vol / avg_cargo_vol), m.floor(limit_mass / avg_cargo_mass))
        cargo_vol = avg_cargo_vol * cargo_pos_count
    
    return cargo_vol / limit_vol * 100.0

def filling_space_percent_from_json_file(json_file_path):
    with open(json_file_path, 'rb') as f:
        cargoes_data = json.load(f)
        f.close()
        return filling_space_percent(cargoes_data)

def estimated_space_percent_from_json_file(json_file_path):
    with open(json_file_path, 'rb') as f:
        cargoes_data = json.load(f)
        f.close()
        return estimated_space_percent(cargoes_data)


In [24]:
import json
from pprint import pprint
from os import path
import glob
import pandas as pd

names = []
total_count = []
unstack_count = []
unturnover_count = []
min_cargo_volume = []
avg_cargo_volume = []
max_cargo_volume = []
min_cargo_mass_ = []
avg_cargo_mass_ = []
max_cargo_mass_ = []
total_cargo_volume = []
limit_cargo_volume = []
total_cargo_mass_ = []
limit_cargo_mass_ = []
filling = []
estimated = []
difference = 0.0
counter = 0

for json_file in glob.glob('ALGORITM/*.json'):
    with open(json_file, 'rb') as f:
        cargoes_data = json.load(f)
        f.close()
        names.append(path.basename(json_file))
        cargoes = cargoes_data['data']['cargoes']
        total_count.append(total_cargo_count(cargoes))
        unstack_count.append(unstackable_cargo_count(cargoes))
        unturnover_count.append(unturnoverable_cargo_count(cargoes))
        min_cargo_volume.append(min_cargo_volume_m3(cargoes))
        avg_cargo_volume.append(weighted_avg_cargo_volume_m3(cargoes))
        max_cargo_volume.append(max_cargo_volume_m3(cargoes))
        min_cargo_mass_.append(min_cargo_mass(cargoes))
        avg_cargo_mass_.append(weighted_avg_cargo_mass(cargoes))
        max_cargo_mass_.append(max_cargo_mass(cargoes))
        total_cargo_volume.append(total_cargo_volume_m3(cargoes))
        limit_cargo_volume.append(limit_volume_m3(cargoes_data['data']['cargo_space']))
        total_cargo_mass_.append(total_cargo_mass(cargoes))
        limit_cargo_mass_.append(cargoes_data['data']['cargo_space']['carrying_capacity'])
        filling.append(filling_space_percent(cargoes_data))
        estimated.append(estimated_space_percent(cargoes_data))
        difference += abs(estimated[-1] - filling[-1])
        counter += 1

cargo_space = pd.DataFrame({'name': names,
                            'cargo_count': total_count,
                            'unstack_cargo_count': unstack_count,
                            'unturnover_cargo_count': unturnover_count,
                            'min_cargo_volume_m3': min_cargo_volume,
                            'avg_cargo_volume_m3': avg_cargo_volume,
                            'max_cargo_volume_m3': max_cargo_volume,
                            'min_cargo_mass': min_cargo_mass_,
                            'avg_cargo_mass': avg_cargo_mass_,
                            'max_cargo_mass': max_cargo_mass_,
                            'total_cargo_volume': total_cargo_volume,
                            'limit_cargo_volume': limit_cargo_volume,
                            'total_cargo_mass': total_cargo_mass_,
                            'limit_cargo_mass': limit_cargo_mass_,
                            'estimated_space_pc': estimated, 'filling_space_pc': filling})
cargo_space.to_csv('cargo-filling-space.csv', index=False, mode='w')

print(difference/counter)
#8.108426910711449

8.108426910711449


### Вывод ###

Сгенерированная таблица показывает, что в отдельных случаях все коробки не помещаются в один грузовой контейнер (`estimated_space_pc > 100.0`). Также замечено, что в некоторых вариантах суммарная масса коробок превышает грузоподьёмность одного транспортного средства. В случаях, когда объём всех коробок меньше объёма контейнера, процент заполненного объёма `estimated_space_pc` совпадает (если все коробки можно кантовать и штабелировать) или очень близок (если есть некантуемые и нештабелируемые коробки) к фактическому проценту заполненного объёма `filling_space_pc`. Среднее отклонение прогноза от фактического заполнения составляет 8%.